In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import model_from_json
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from matplotlib import pyplot as plt
import os
import cv2
from random import shuffle

In [ ]:
# Change directories here
mainDir = "/content/drive/My Drive/pokeData/Final Project Combined"
folder = "cropped_data"

# Change between "training" (for no augmented data) and "newTraining" (for augmented data)
typeO = "newTraining"

# The best weight file and model saved within the directory
filepath=os.path.join(mainDir, folder, typeO, "weights.best.h5")
modelPath = os.path.join(mainDir, folder, typeO, "model.json")
weightFile = os.path.join(mainDir, folder, typeO, "weights.h5")

In [ ]:
DATADIR = os.path.join(mainDir, folder, typeO)
CATEGORIES_train = ["pikachu", "bulbasaur", "squirtle", "charmander", "meowth"]

In [ ]:
# Image resized to 224x224x3 that is the ResNet50 input requirements
IMG_SIZE = 224
training_data = []

def create_training_data():
  """
  Function reads through all the available images in each Pokemon directory,
  resizes according to requirement and appends to training_data along with the class number.
  """
  for category in CATEGORIES_train:
    path = os.path.join(DATADIR, category)
    class_num = CATEGORIES_train.index(category)
    for img in os.listdir(path):
      try:
        img_array = cv2.imread(os.path.join(path, img))
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        training_data.append([new_array, class_num])
      except Exception as e:
        pass

create_training_data()

In [ ]:
X = []
Y = []

# Training data is shuffled so that the network gets random images and
# not each pokemon in a sequence.
shuffle(training_data)

In [ ]:
# Splitting the image data and the label to two separate lists.
for features, label in training_data:
  X.append(features)
  Y.append(label)

In [ ]:
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y = np.array(Y)

In [ ]:
# Downloading the ResNet50 model comprising of all the weights from the ImageNet database
pretrained_model = ResNet50(input_shape = [IMG_SIZE, IMG_SIZE, 3], weights='imagenet', include_top = False)
pretrained_model.trainable = False

In [ ]:
# Designing the layers of our model
model = keras.Sequential([
                          pretrained_model,
                          keras.layers.Flatten(),
                          keras.layers.Dense(5, activation=tf.nn.softmax)
])

In [ ]:
# Compiling our model with Adam optimizer and crossentropy loss function
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Creating a checkpoint function on the basis of validation accuracy of the model
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# An increase in the validation accuracy makes the program save the new weight file
# The best weight file is then used in the testing process
callbacks_list = [checkpoint]

In [ ]:
# Fitting the model using 33% of dataset as Validation data
model.fit(X, Y, validation_split=0.33, epochs=50, callbacks=callbacks_list, verbose=1)

Epoch 1/50
36/36 [==============================] - ETA: 0s - loss: 3.5758 - accuracy: 0.7908
Epoch 00001: val_accuracy improved from -inf to 0.96655, saving model to /content/drive/My Drive/pokeData/Final Project Combined/cropped_data/newTraining/weights.best.h5
36/36 [==============================] - 231s 6s/step - loss: 3.5758 - accuracy: 0.7908 - val_loss: 0.3453 - val_accuracy: 0.9665
Epoch 2/50
36/36 [==============================] - ETA: 0s - loss: 0.0678 - accuracy: 0.9913
Epoch 00002: val_accuracy improved from 0.96655 to 0.98063, saving model to /content/drive/My Drive/pokeData/Final Project Combined/cropped_data/newTraining/weights.best.h5
36/36 [==============================] - 236s 7s/step - loss: 0.0678 - accuracy: 0.9913 - val_loss: 0.2067 - val_accuracy: 0.9806
Epoch 3/50
36/36 [==============================] - ETA: 0s - loss: 0.0488 - accuracy: 0.9931
Epoch 00003: val_accuracy did not improve from 0.98063
36/36 [==============================] - 235s 7s/step - loss

In [ ]:
# Making model and weight files so that test can access it
model_json = model.to_json()
with open(modelPath, "w") as json_file:
    json_file.write(model_json)
# Serialize weights to HDF5
model.save_weights(weightFile)
print("Saved model and weights to disk")

In [ ]:
%reset